## Question 1, Week - 3

First, import the libs well need. then use wget to pull down the wikipedia page about Toroto Neighborhoods


For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

    Start by creating a new Notebook for this assignment.
    Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis 
import codecs 

!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print("Done")

--2019-12-28 13:54:54--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78673 (77K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M.3’

List_of_postal_code 100%[===================>]  76.83K  --.-KB/s    in 0.03s   

2019-12-28 13:54:54 (2.53 MB/s) - ‘List_of_postal_codes_of_Canada:_M.3’ saved [78673/78673]

Done


In [3]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install -c anaconda beautifulsoup4

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests 
from bs4 import BeautifulSoup
print('Imported BeautifulSoup')

Imported BeautifulSoup


In [13]:
conda install -c anaconda lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.3.0-py36hefd8a0e_0 anaconda


lxml-4.3.0           | 1.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.




In [6]:
f = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = f.read()
soup = BeautifulSoup(html, 'html.parser')

df = (pd.read_html(str(soup.table))[0])

#drop the first row, which is the headings of the table
df.drop(axis=0, index=0, inplace=True)

#then assign the proper column names
column_names = ['Postcode', 'Borough', 'Neighborhood'] 
df.columns = column_names

#get rid of unassigned Boroughs
df = df[df['Borough'] != "Not assigned"].reset_index(drop=True) 
df

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [7]:
na = df[df['Neighborhood']=="Not assigned"]
for index in range(len(na)):
    r = na.iloc[index]
    r['Neighborhood']=r['Borough']
    df.iloc[na.index[0]]['Neighborhood'] = r['Borough']

df.head(8)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge


In [11]:
aggregations = {
    'Neighborhood':lambda x:', '.join(x)}
g = df.groupby('Postcode').agg(aggregations).reset_index()
x = df.drop('Neighborhood',axis=1)
m = pd.merge(x, g, on='Postcode', how='left')
dd = m.drop_duplicates()
dd.to_csv("Toronto_Neighborhood.csv", index=False)
dd # dd means dropped duplicates

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Queen's Park,Queen's Park
...,...,...,...
192,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
195,M4Y,Downtown Toronto,Church and Wellesley
196,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
197,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [12]:
dd.shape

(103, 3)